In [1]:
    var sparkConf = new SparkConf().setAppName("Gradient Boosting Regression");
    var sc = new SparkContext(sparkConf);


 Load and parse the data file.


In [2]:
var datapath =  "../data/mllib/sample_libsvm_data.txt";
    var data = MLUtils.loadLibSVMFile(sc, datapath);


 Split the data into training and test sets (30% held out for testing)


In [3]:
    var splits = data.randomSplit([0.7, 0.3]);
    var trainingData = splits[0];
    var testData = splits[1];


 Train a GradientBoostedTrees model.
 The defaultParams for Regression use SquaredError by default.


In [4]:
    var boostingStrategy = BoostingStrategy.defaultParams("Regression");
    boostingStrategy.setNumIterations(3); // Note: Use more iterations in practice.
    boostingStrategy.getTreeStrategy().setMaxDepth(5);


 Empty categoricalFeaturesInfo indicates all features are continuous.


In [5]:
    var categoricalFeaturesInfo = {};
    boostingStrategy.getTreeStrategy().setCategoricalFeaturesInfo(categoricalFeaturesInfo);

    var model = GradientBoostedTrees.train(trainingData, boostingStrategy);


In [6]:
    var predictionAndLabel = testData.mapToPair(function (lp, model) {
        return new Tuple(model.predict(lp.getFeatures()), lp.getLabel());
    }, [model]);


In [7]:
    var testMSE = predictionAndLabel.map(function (tuple2) {
            var diff = parseFloat(tuple2[0] - tuple2[1]);
            return diff * diff;
        }).reduce(function (a, b) {
            return a + b;
        }) / data.count();

    var ret = {};
    ret.testMSE = testMSE;
    ret.model = model;
var result = ret;
    print("Test Mean Squared Error: " + result.testMSE);
    print("Learned regression GBT model:\n" + result.model.toDebugString());


 Save and load model


In [8]:
    result.model.save(sc, "target/tmp/myGradientBoostingRegressionModel");
    var sameModel = GradientBoostedTreesModel.load(sc, "target/tmp/myGradientBoostingRegressionModel");

    sc.stop();
